# Web Scraping in Python Series

## Introduction
The system will provide a simple example of using python to extract information from a website. Note this is part 1 of 4. Participants are not expected to have any experience in python or background in web scraping. However, some understanding of HTML will be useful.

This draws inspiration from http://web.stanford.edu/~zlotnick/TextAsData/Web_Scraping_with_Beautiful_Soup.html

In [9]:
# Import dependencies (i.e. packages that extend the standard language to perform specific [advance] functionality)
import urllib
from datetime import datetime, date, timedelta
from bs4 import BeautifulSoup

From analysing the newsday archieve website we see that the URL follows a parsable convention
http://www.newsday.co.tt/archives/YYYY-M-DD.html 
So our general approach will be as follows:
    1. Generate date in the expected form between an ending and starting date
    2. Test to ensure the dates generated are valid. (refine step1 based on results)
    3. Read the content and process based on our goal for scaping the page

In [10]:
# Step 1 - create a function to generates a list(array) of dates
def genDates(start_date = date.today(), num_days = 3):
    # date_list = [start - timedelta(days=x) for x in range(0, num_days)] # generate a list of dates
    # While we expand the above line for beginners understanding 
    date_list = []
    for d in range(0, num_days):
        temp = start_date - timedelta(days=d)
        date_list.append(temp.strftime('%Y-%-m-%d'))# http://strftime.org/ used a reference
    return date_list

In [26]:
# Step 2 -Test the generated URL to ensure they point to 
def traverseDates(func, start_date = date.today(), num_days = 3):
    base_url="http://www.newsday.co.tt/archives/"
    dates_str_list = genDates(start_date, num_days)
    for date in dates_str_list:
        url = base_url + date
        func(url)
        
def printDate(date):
    print(date)
    
traverseDates(printDate)

http://www.newsday.co.tt/archives/2017-2-02
http://www.newsday.co.tt/archives/2017-2-01
http://www.newsday.co.tt/archives/2017-1-31


In [12]:
# Step 3 - Read content and process page
def processPage(page_url):
    print("Attempting to read content from {0}".format(page_url))
    page_content = urllib.urlopen(page_url).read()
    beau = BeautifulSoup(page_content, "html5lib")
    tables = beau.find_all("table") #https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-all
    for i in range(0,13):
        named_sec = tables[i].h3
        if named_sec:
            print("i {0} produced {1}".format(i,named_sec))
    article_links = beau.find_all("a", 'title')
    print("Found {0} tables and {1} articles".format(len(tables), len(article_links)))

In [13]:
# traverseDates(processPage,num_days = 1)


## The Purpose (Goal) of Scraping
Our main purpose of developing this exercise was to determine if the statement that the majority of the news published was negative. To do this we need to capture the sentiment of the information extracted from the link. While we can develop sentiment analysis tools using python, the process of training and validating is too much work at this time. Therefore, we utilize the IBM Watson Tone Analyzer API. We selected this API because it provides a greater amount of detail rather than a binary positive or negative result.

To use the watson api for python:

1. We installed the pip pacakge
    ```bash
    pip install --upgrade watson-developer-cloud
    ```
2. We created an account (free for 30 days)
    https://tone-analyzer-demo.mybluemix.net/
    
3. Use the API referene to build application
    http://www.ibm.com/watson/developercloud/tone-analyzer/api/v3/?python#
4. Created a local_settings.py file that contains the credentials retrieved from signing up

In [14]:
# Integrating IBM Watson
import json
from watson_developer_cloud import ToneAnalyzerV3
from local_settings import *

In [15]:
def getAnalyser():
    tone_analyzer = ToneAnalyzerV3(
       username= WATSON_CREDS['username'],
       password= WATSON_CREDS['password'],
       version='2016-05-19')
    return tone_analyzer


In [16]:
# tone_analyzer = getAnalyser()
# tone_analyzer.tone(text='A word is dead when it is said, some say. Emily Dickinson')

In [23]:
def analysePage(page_url):
    page_content = urllib.urlopen(page_url).read()
    beau = BeautifulSoup(page_content, "html5lib")
    tables = beau.find_all("table") #https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-all
    article_links = beau.find_all("a", 'title')
    print("Found {0} tables and {1} articles".format(len(tables), len(article_links)))
    for i in article_links:
        print i

In [25]:
# traverseDates(analysePage,num_days = 1)

In [40]:
page_content = urllib.urlopen("http://www.newsday.co.tt/archives/2017-2-2").read()
beau = BeautifulSoup(page_content, "html5lib")
tables = beau.find_all("table") #https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-all
article_links = beau.find_all("a", 'title')
print("Found {0} tables and {1} articles".format(len(tables), len(article_links)))

Found 13 tables and 39 articles


In [41]:
def processTone(tone):
    large = tone[0]['score']
    large_i = 0
    for i in range(1, len(tone)):
        if tone[i]['score'] > large:
            large = tone[i]['score']
            large_i = i
    return tone[large_i]

The Understanding of the structure of the response is provided in the API reference

https://www.ibm.com/watson/developercloud/tone-analyzer/api/v3/?python#post-tone

In [42]:
first = True
emo_count = {
    "anger" : 0,
    "disgust": 0,
    "fear" : 0,
    "joy" : 0,
    "sadness": 0
}
socio_count = {
    "openness_big5": 0,
    "conscientiousness_big5": 0,
    "extraversion_big5" : 0,
    "agreeableness_big5" : 0,
    "emotional_range_big5": 0
}
for i in article_links:
    res = tone_analyzer.tone(i['title'])
    tone = res['document_tone']['tone_categories']
    emo = tone[0]['tones'] # we want the emotional tone
    soci= tone[2]['tones'] # we also want the social tone
    
    e_res = processTone(emo)
    emo_count[e_res['tone_id']] += 1
    
    s_res = processTone(soci)
    socio_count[s_res['tone_id']] += 1

In [43]:
for e in emo_count:
    print("{0} articles were classified with the emotion {1}".format(emo_count[e], e))

3 articles were classified with the emotion anger
11 articles were classified with the emotion joy
4 articles were classified with the emotion fear
15 articles were classified with the emotion sadness
6 articles were classified with the emotion disgust


In [44]:
for s in socio_count:
    print("{0} articles were classified as {1}".format(socio_count[s], s))

11 articles were classified as extraversion_big5
5 articles were classified as openness_big5
14 articles were classified as conscientiousness_big5
5 articles were classified as agreeableness_big5
4 articles were classified as emotional_range_big5
